In [95]:
"""
Hoi Maarten! Hierbij een update over mijn progressie tot dusver. Afgelopen week heb ik veel moeite gehad
met het ondersteunen van verschillende soorten PDF. Zoals ik al eerder heb aangegeven is PDF heel uitgebreid en
kan tekst op heel veel verschillende manieren samengesteld zijn.

De te verwijderen tekst uit verschillende soorten documenten halen was de grootste uitdaging van deze week en ik 
heb heel veel tijd besteed aan verschillende oplossingen te bedenken. Uiteindelijk kwam ik terecht bij het punt
dat mijn oorspronkelijke methode niet adequaat was voor het afhandelen van alle cases. In theorie, met veel meer 
tijd, was het zeker mogelijk geweest. De verwijder methode in de PyMuPdf library was echter wel adequaat omdat 
deze al veel ondersteunde wat ik zelf in deze drie maanden onmogelijk zelf kon maken. Ik ben overgestapt op deze 
methode.

Echter ontbreekt er een belangrijk onderdeel voor het verplaatsen van tekst in PDF documenten: dit wordt namelijk
niet ondersteund in PyMuPdf. Hiervoor heb ik mijn eigen algoritme moeten schrijven en deze lijkt al veel gevallen 
goed op te lossen, echter moet er wat betreft de details nog aardig wat gebeuren en moet er nog flink getest mee 
worden. 

Ik heb vandaag een klein voorbeeldje voor je.
"""

'\nHoi Maarten! Hierbij een update over mijn progressie tot dusver. Afgelopen week heb ik veel moeite gehad\nmet het ondersteunen van verschillende soorten PDF. Zoals ik al eerder heb aangegeven is PDF heel uitgebreid en\nkan tekst op heel veel verschillende manieren samengesteld zijn.\n\nDe te verwijderen tekst uit verschillende soorten documenten halen was de grootste uitdaging van deze week en ik \nheb heel veel tijd besteed aan verschillende oplossingen te bedenken. Uiteindelijk kwam ik terecht bij het punt\ndat mijn oorspronkelijke methode niet adequaat was voor het afhandelen van alle cases. In theorie, met veel meer \ntijd, was het zeker mogelijk geweest. De verwijder methode in de PyMuPdf library was echter wel adequaat omdat \ndeze al veel ondersteunde wat ik zelf in deze drie maanden onmogelijk zelf kon maken. Ik ben overgestapt op deze \nmethode.\n\nEchter ontbreekt er een belangrijk onderdeel voor het verplaatsen van tekst in PDF documenten: dit wordt namelijk\nniet onderst

In [171]:
import fitz
import sys
import re
sys.path.append('../src/safePdfRedactionTool')
from DocumentRedactor import DocumentRedactor

In [172]:
"""
Neem een simpel voorbeeld pdfje

p.s. Misschien moet u de pagina van de link refreshen om de pdf te zien!
"""
from IPython.display import display_pdf
with open('../resources/testpdf/test2.pdf', "rb") as f:
    display_pdf(f.read(),raw=True)


In [173]:
"""
Dit document gaan we inladen en uitlezen
"""

redactor = DocumentRedactor("/home/lennaert/Thesis-Lennaert-Feijtes-Safe-PDF-Redaction-Tool/resources/testpdf/test2.pdf")
pages = redactor.get_pages()

In [174]:
"""
Laten we in dit geval het eerste woordje pakken op de PDF en die weghalen.
"""

for page in pages:
    redactor.prepare_page(page)
    dim = redactor.get_page_dimensions(page)
    (xref, lines, words) = redactor.get_page_contents(page)
    redactor.remove_positional_adjustments(lines, xref)

    # Een test weglak van een woord op dezelfde lijn
    redactions = [words[2]] # Eerste woord op de PDF
    first_redaction = redactions[0]
    for redaction in redactions:
        rect = fitz.Rect(redaction[0], redaction[1], redaction[2], redaction[3])
        redactor.add_redact_annot(page, rect)

    redactor.doc.save("res_temp.pdf")
    redactor.apply_redactions(page)

In [175]:
"""
In dit tussentijdse voorbeeld is te zien wat we willen gaan weghalen.
"""

with open('res_temp.pdf', "rb") as f:
    display_pdf(f.read(),raw=True)


In [176]:
"""
We vervangen nu dit weggelakte met een "x".
"""

for page in pages:    
    (xref, lines, words) = redactor.get_page_contents(page)

    # insert text "x" in first redaction
    test = fitz.Rect(first_redaction[0], first_redaction[1]-2, first_redaction[2], first_redaction[3]+5)
    test = redactor.insert_replacement_text(page, test)
    redactor.doc.save("res_temp2.pdf")

    (xref, lines, words) = redactor.get_page_contents(page)
    
    with open('res_temp2.pdf', "rb") as f:
        display_pdf(f.read(),raw=True)



Rect(101.5050048828125, 70.056396484375, 106.7740478515625, 90.484130859375) 107.5050048828125


In [177]:
"""
Het bepalen van de nieuwe posities voor complexe manieren van tekst renderen in een PDF moet nog 
afgewerkt worden. Voornamelijk de positie bepaling. Niet alle gevallen doen het nog en soms is de 
afstand tussen woorden nog niet juist. 

Voor documenten die minder complexe manieren van tekst renderen gebruiken is het veranderen van plaatsen
een stuk makkelijker en die doet het al aardig goed!
"""

for page in pages:
    pattern = re.compile(r'<[^>]+>|\([^)]+\)')
    for i in range(len(lines)):
        if lines[i].endswith(b"TJ"):
            if b"Tm" not in lines[i] or b"Tf" not in lines[i]:
                #print(lines[i])
                operands = lines[i].strip()[:-2].decode().strip()[1:-1]
                res = re.sub(pattern, " ", operands).strip().split()
                new = operands
                if len(res) > 0:
                    for r in range(len(res)):
                        if r > 0:
                            new = new.replace(res[r], "-1")
                            operands = new
                        else:
                            #TODO calculate based on text
                            # Hieraan moet nog gewerkt worden
                            new = new.replace(res[r], "-800")
                            operands = new

                    lines[i] = b"[" + operands.encode() + b"]" + b" TJ"
                #print("ok", lines[i])


    to_be_repositioned = redactor.get_to_be_repositioned_words(dim[1], lines, first_redaction, [test])
    redactor.reposition_words_same_line(to_be_repositioned, redactions, [test], lines, xref)

    redactor.finalize_redactions()

with open('res.pdf', "rb") as f:
        display_pdf(f.read(),raw=True)

Rect(101.5050048828125, 70.056396484375, 107.5050048828125, 106.7740478515625) 72.025 82.52999999999997
Rect(101.5050048828125, 70.056396484375, 107.5050048828125, 106.7740478515625) 124.55 82.52999999999997
Rect(101.5050048828125, 70.056396484375, 107.5050048828125, 106.7740478515625) 72.025 105.02999999999997
Rect(101.5050048828125, 70.056396484375, 107.5050048828125, 106.7740478515625) 125.8 105.02999999999997


In [ ]:
"""
Hier een voorbeeld van meer weglakken!
"""